In [1]:
# from sidecar import Sidecar
from ipywidgets import IntSlider, Layout
from ipyleaflet import Map, basemaps, Marker, projections
import xarray as xr
import numpy as np
# import xarray_leaflet
import matplotlib.pyplot as plt
import rioxarray
# import odc.ui
# import hvplot
import leafmap.leafmap as leafmap
# import os
# os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = f'{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}'

## Load data

In [ ]:
filename = '/g/data/jk72/MIZ/processed/monthly_sea_ice_edge_AMSR2012-2020.nc'

data = xr.open_dataset(filename)

# monthly_average = data.groupby('time.month').mean('time')

# monthly_average = monthly_average.rename_dims({'month':'time'})

In [ ]:
data.isel(time=100).drop('time').rename_dims({'x':'lon','y':'lat'}).to_netcdf('/g/data/jk72/MIZ/processed/singleedge.nc')

In [ ]:
monthly_average = monthly_average.rename_dims({'x':'lon','y':'lat'})
monthly_average.to_netcdf('/g/data/jk72/MIZ/Bremen/netcdf/monthly_average_asi.nc')

In [ ]:
jan = monthly_average['conc'].isel(time=8).to_netcdf('/g/data/jk72/MIZ/Bremen/netcdf/monthly_average_sep_asi.nc')

## Create tracker interface

In [ ]:
# m = Map(center=[40, 115], zoom=3, basemap=basemaps.CartoDB.DarkMatter, interpolation='nearest')

In [ ]:
# South Polar sterographic
spsLayout=Layout(width='800px', height='800px')

m = Map(center=(-90, 0),
        zoom=0,
        layout=spsLayout,
        basemap=basemaps.NASAGIBS.BlueMarble3031,
        crs=projections.EPSG3031)

# sc = Sidecar(title='Sidecar Output')

# with sc:
display(m)
# l = jan.leaflet.plot(m, colormap=plt.cm.terrain)


In [ ]:
# Expects an RGB input, not a single band like conc. Needs modifying to work.

# img_layer = odc.ui.mk_image_overlay(
#     jan,
#     clamp=2000,  # 2000 -- brightest pixel level
#     bands=['conc'],
#     fmt='png')   # "jpeg" is another option

# Add image layer to a map we created earlier
# m.add_layer(img_layer)

In [ ]:
center = (-70, 70)

m = Map(center=center, zoom=5)

marker = Marker(location=center, draggable=True)
m.add_layer(marker);


# Now that the marker is on the Map, you can drag it with your mouse,
# it will automatically update the `marker.location` attribute in Python

# You can also update the marker location from Python, that will update the
# marker location on the Map:
marker.location = (50, 356)

sl = IntSlider(description='Some slider')
with sc:
#     display(sl)
    display(m)

## Leafmap

In [3]:
data = xr.open_dataset('/g/data/jk72/MIZ/processed/DEA GeoMAD (Landsat 8 OLI-TIRS) clip.tiff', engine='rasterio').isel(band=1)

In [4]:
data.geo

<xarray.Dataset>
Dimensions:      (x: 716, y: 796)
Coordinates:
    band         int64 2
  * x            (x) float64 1.447e+06 1.447e+06 ... 1.468e+06 1.468e+06
  * y            (y) float64 -4.057e+06 -4.057e+06 ... -4.081e+06 -4.081e+06
    spatial_ref  int64 ...
Data variables:
    band_data    (y, x) float32 ...

In [ ]:
# South Polar sterographic
spsLayout=Layout(width='800px', height='800px')

m = leafmap.Map()

data = '/g/data/jk72/MIZ/processed/DEA GeoMAD (Landsat 8 OLI-TIRS) clip.tiff'
m.add_geotiff(data, bands=[1], layer_name='conc')

# url = 'https://seaice.uni-bremen.de/data/amsr2/asi_daygrid_swath/s6250/2015/jan/Antarctic/asi-AMSR2-s6250-20150101-v5.4.tif'
# leafmap.cog_bounds(url)
# leafmap.cog_center(url)
# leafmap.cog_bands(url)
# leafmap.cog_tile(url)
# m.add_cog_layer(url, name="sea ice conc")
# 

m

In [ ]:
# South Polar sterographic
spsLayout=Layout(width='800px', height='800px')

m = leafmap.Map(center=(-90, 0),
        zoom=0,
        layout=spsLayout,
        basemap=basemaps.NASAGIBS.BlueMarble3031,
        crs=projections.EPSG3031)

data = '/g/data/jk72/MIZ/Bremen/geotiff/asi-AMSR2-s6250-20180505-v5.4.tif'
m.add_geotiff(data, bands=[1], layer_name='conc')

# url = 'https://seaice.uni-bremen.de/data/amsr2/asi_daygrid_swath/s6250/2015/jan/Antarctic/asi-AMSR2-s6250-20150101-v5.4.tif'
# leafmap.cog_bounds(url)
# leafmap.cog_center(url)
# leafmap.cog_bands(url)
# leafmap.cog_tile(url)
# m.add_cog_layer(url, name="sea ice conc")
# 

# filename = 'DEA GeoMAD (Landsat 8 OLI-TIRS) clip.tiff'
# filename = 'asi-AMSR2-s6250-20150101-v5.4.tif'
# filename = 'S_201905_concentration_v3.0.tif'
# m.add_raster(filename, bands=[1], layer_name='conc')
# m.add_geotiff(filename, layer_name='conc')
# m.add_netcdf('DEA GeoMAD (Landsat 8 OLI-TIRS) clip.nc')
# m.add_local_tile(filename)

# url = 'https://seaice.uni-bremen.de/data/amsr2/asi_daygrid_swath/s6250/2015/jan/Antarctic/asi-AMSR2-s6250-20150101-v5.4.tif'
# url = 'ttps://masie_web.apps.nsidc.org/pub//DATASETS/NOAA/G02135/'
# leafmap.cog_bounds(url)
# leafmap.cog_center(url)
# leafmap.cog_bands(url)
# leafmap.cog_tile(url)
# m.add_cog_layer(url, name="sea ice conc")
# m.add_remote_tile(url)
# m.add_tile_layer(url, name = 'a', attribution='y')

m

In [2]:
# LEAFMAP WMS

# South Polar sterographic
spsLayout=Layout(width='900px', height='1200px')

POLAR3031 = dict(
    name='EPSG:3031',
    custom=True,
    proj4def="""+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1
        +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs""",
    bounds =[[-2822131.5,-3057369.25],[3744213.75,3822194.25]]
)

m = leafmap.Map(
    center=(-90, 0),
        zoom=1,
        layout=spsLayout,
        basemap=basemaps.NASAGIBS.BlueMarble3031,
        crs=projections.EPSG3031)

m.add_wms_layer(url='http://geos.polarview.aq/geoserver/wms', layers='polarview:iceedgeS15', format='image/png',  transparent=True, attribution='Polarview', crs=POLAR3031)

m

Map(center=[-90, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

## IPYLEAFLET

In [8]:
import numpy as np
from ipyleaflet import Map, basemaps, ImageOverlay, projections
import rioxarray as rio
from ipywidgets import Layout
import xarray as xr
import odc.geo.xr

def polar_map(da, epsg):
    da = da.odc.reproject(epsg)
    vmin, vmax = np.nanpercentile(da.data, [2, 98])
    rgba = da.odc.colorize(vmin=vmin, vmax=vmax)
    data_url = rgba.odc.compress(as_data_url=True)
    (x1, y1), _, (x2, y2) = rgba.odc.geobox.extent.exterior.to_crs("epsg:4326").points[:3]
    bounds = [[y1, x1], [y2, x2]]
    return data_url, bounds

spsLayout=Layout(width='800px', height='800px')

path = '/g/data/jk72/sc0554/miz/data/S_201905_concentration_v3.0.tif'

yy = rio.open_rasterio(path, mask=True).isel(band=0)

yy = xr.where(yy > 2500, np.nan, yy, keep_attrs=True)

data_url, bounds = polar_map(yy, epsg="EPSG:3031")

m = Map(basemap=basemaps.NASAGIBS.BlueMarble3031, center=(-90, 0), zoom=1, crs=projections.EPSG3031, layout=spsLayout)

m.add_layer(ImageOverlay(url=data_url, bounds=bounds, opacity=0.7))

m

Map(center=[-90, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [9]:
from ipyleaflet import Map, WMSLayer, basemaps

spsLayout=Layout(width='800px', height='800px')

POLAR3031 = dict(
    name='EPSG:3031',
    custom=True,
    proj4def="""+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1
        +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs""",
    bounds =[[-2822131.5,-3057369.25],[3744213.75,3822194.25]]
)

wms = WMSLayer(
    url='http://geos.polarview.aq/geoserver/wms',
    layers='polarview:iceedgeS15',
    format='image/png',
    transparent=True,
    attribution='Polarview',
    crs=POLAR3031
)

m = Map(basemap=basemaps.NASAGIBS.BlueMarble3031, center=(-90, 0), zoom=1, crs=projections.EPSG3031, layout=spsLayout)

m.add_layer(wms)

m

Map(center=[-90, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

## Xarray leaflet

In [ ]:
from xarray_leaflet.transform import passthrough
from ipyleaflet import Map
import xarray_leaflet

In [ ]:
da = rioxarray.open_rasterio('S_201905_concentration_v3.0.tif')
da = da.isel(band=0).drop('band'

In [ ]:
m = Map()
l = da.leaflet.plot(m, transform1=passthrough)
m

## HVPLOT

In [ ]:
import hvplot.xarray

In [ ]:
data = xr.open_dataset('/g/data/jk72/MIZ/processed/asi2_monthly.nc')

In [ ]:
data=data.isel(time=slice(0,3))

In [ ]:
data['conc'].hvplot(groupby='time',frame_height=500, frame_width=500)